In [0]:
%run "../commons/common_functions"

In [0]:
%run "../commons/configuration"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import to_timestamp, concat, lit, col
from pyspark.sql.types import *


qualifying_schema = StructType([StructField("qualifyId", IntegerType(), False), 
                            StructField("raceId", IntegerType(), True), 
                            StructField("driverId", IntegerType(), True), 
                            StructField("constructorId", IntegerType(), True), 
                            StructField("number", IntegerType(), True),
                            StructField("position", IntegerType(), True),
                            StructField("q1", StringType(), True),
                            StructField("q2", StringType(), True),
                            StructField("q3", StringType(), True),
                            ])

qualifying_df = spark.read.json(f"{raw_mount_path}/qualifying/", schema=qualifying_schema, multiLine=True)\
    .withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumnRenamed("constructorId", "constructor_id")\
    .withColumnRenamed("qualifyId", "qualify_id")\
    .withColumn("data_source",lit(p_env_name))


In [0]:
qualifying_df_v2 = add_ingestion_date(qualifying_df)

In [0]:
#qualifying_df_v2.write.parquet(f"{processed_mount_path}/qualifying", mode="overwrite")
qualifying_df_v2.write.mode("overwrite").format("parquet").saveAsTable("processed.qualifying")

In [0]:
dbutils.notebook.exit("Success")